In [1]:
import os
import h5py
import argparse
import numpy as np

In [2]:
def parse_args():
    parser = argparse.ArgumentParser(
        description="Merges numpy arrays; outputs hdf5 file")
    parser.add_argument("input_file_list",
                        type=str, nargs=1,
                        help="Path to input text file,\
                        each file on a different line.")
    parser.add_argument('output_file', type=str, nargs=1,
                        help="Path to output file.")  
    args = parser.parse_args()
    return args

def count_events(files):
    num_events = 0
    for f in files:
        data = np.load(f)
        num_events += data['event_id'].shape[0]
    return num_events


In [3]:
EVENT_SHAPE = (15808, 2)

In [ ]:
config = parse_args()

#read in the input file list
with open(config.input_file_list[0]) as f:
    files = f.readlines()

#remove whitespace 
files = [x.strip() for x in files] 

# -- Check that files were provided
if len(files) == 0:
    raise ValueError("No files provided!!")
print("Merging "+str(len(files))+" files")

num_events = count_events(files)


In [4]:
files = ["IWCDmPMT_4pi_full_tank_e-_E0to1000MeV_unif-pos-R371-y521cm_4pi-dir_3000evts_518.npz"]
num_events = count_events(files)
print(num_events)

3000


In [5]:
# Only use 32 bit precision
dtype_events = np.dtype(np.float32)
dtype_labels = np.dtype(np.int32)
dtype_energies = np.dtype(np.float32)
dtype_positions = np.dtype(np.float32)
dtype_IDX = np.dtype(np.int32)
dtype_PATHS = h5py.special_dtype(vlen=str)
dtype_angles = np.dtype(np.float32)

# Initialize HDF5
# h5_file = h5py.File(config.output_file[0], 'w')
h5_file = h5py.File("test.h5", 'w')
dset_event_data = h5_file.create_dataset("event_data",
                                   shape=(num_events,)+EVENT_SHAPE,
                                   dtype=dtype_events)
dset_labels = h5_file.create_dataset("labels",
                               shape=(num_events,),
                               dtype=dtype_labels)
dset_energies = h5_file.create_dataset("energies",
                                 shape=(num_events, 1),
                                 dtype=dtype_energies)
dset_positions = h5_file.create_dataset("positions",
                                  shape=(num_events, 1, 3),
                                  dtype=dtype_positions)
dset_IDX = h5_file.create_dataset("event_ids",
                            shape=(num_events,),
                            dtype=dtype_IDX)
dset_PATHS = h5_file.create_dataset("root_files",
                              shape=(num_events,),
                              dtype=dtype_PATHS)
dset_angles = h5_file.create_dataset("angles",
                             shape=(num_events, 2),
                             dtype=dtype_angles)

In [6]:
offset = 0

for filename in files:
    data = np.load(filename, allow_pickle=True)
    
    # Meta data of file
    num_events_in_file = len(data['digi_hit_pmt'])
    x_data = np.zeros((num_events_in_file,)+EVENT_SHAPE, 
                      dtype=dtype_events)
    digi_hit_pmt = data['digi_hit_pmt']
    digi_hit_charge = data['digi_hit_charge']
    digi_hit_time = data['digi_hit_time']
    
    # Storing the events
    for i in range(num_events_in_file):
        hit_pmts = digi_hit_pmt[i]
        charge = digi_hit_charge[i]
        time = digi_hit_time[i]
        for j in range(len(hit_pmts)):
            ### Parsing the data ###
            hit_pmt = hit_pmts[j]
            x_data[i, hit_pmt, 0] = charge[j]
            x_data[i, hit_pmt, 1] = time[j]
    
    # More meta data
    directions = data['direction']
    if 'IWCDmPMT_4pi_full_tank_gamma' in filename:
        labels = np.zeros(num_events_in_file)
        e_mass=0.51099895000
        energies=np.expand_dims(data['energy'],1)
        mag_momenta=np.sqrt(energies**2-e_mass**2)
        mag_momenta=np.expand_dims(mag_momenta,2)
        momenta=mag_momenta*directions
        momenta_sum=np.sum(momenta,axis=1)
        momenta_sum_adj=momenta_sum[:,[2,0,1]]
        momenta_sum_adj_mag=np.sqrt(np.sum(momenta_sum_adj**2,axis=1))
        directions_proper=momenta_sum_adj/np.expand_dims(momenta_sum_adj_mag,1)
        energies = np.sum(energies, axis=1).reshape(-1,1)
        positions = np.expand_dims(data['position'],1)[:,0,:].reshape(-1, 1,3)
    elif 'IWCDmPMT_4pi_full_tank_e-' in filename:
        labels = np.ones(num_events_in_file)
        energies = np.expand_dims(data['energy'],1)
        positions = np.expand_dims(data['position'],1)
        directions = directions.squeeze()
        directions_proper = directions[:,[2,0,1]]
    elif 'IWCDmPMT_4pi_full_tank_mu-' in filename:
        labels = np.full(num_events_in_file, 2)
        energies = np.expand_dims(data['energy'],1)
        positions = np.expand_dims(data['position'],1)
        directions = directions.squeeze()
        directions_proper = directions[:,[2,0,1]]
    else:
        raise ValueError("File {} is not electron, muon or gamma".format(filename))
    energies = energies.astype(dtype_energies)
    positions = positions.astype(dtype_positions)
    planar_mag=np.sqrt(np.sum(directions_proper[:,[0,1]]**2,axis=1))
    azimuthal=np.arctan2(directions_proper[:,1],directions_proper[:,0])
    polar=np.arctan2(directions_proper[:,2],planar_mag)
    azimuthal=azimuthal.reshape(-1,1)
    polar=polar.reshape(-1,1)
    angles=np.hstack((polar,azimuthal))

    PATHS = data['root_file'].astype(dtype_PATHS)
    IDX = data['event_id'].astype(dtype_IDX)

    offset_next = offset + num_events_in_file

    dset_event_data[offset:offset_next,:] = x_data
    dset_labels[offset:offset_next] = labels
    dset_energies[offset:offset_next,:] = energies
    dset_positions[offset:offset_next,:,:] = positions
    dset_PATHS[offset:offset_next] = PATHS
    dset_IDX[offset:offset_next] = IDX
    dset_angles[offset:offset_next,:] = angles

    offset = offset_next
    print("Finished file: {}".format(filename))

Finished file: IWCDmPMT_4pi_full_tank_e-_E0to1000MeV_unif-pos-R371-y521cm_4pi-dir_3000evts_518.npz


In [7]:
print("Saving")
h5_file.close()
print("Finished")

Saving
Finished
